In [1]:
pip install scipy pillow matplotlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install h5py


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install geopandas rasterio shapely tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
import rasterio
import geopandas as gpd
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import warnings
import cv2
import h5py
from shapely.geometry import box
import pandas as pd

# Data processing 
- Turkey buildings: https://data.humdata.org/dataset/hotosm_tur_buildings 
- Turkey destroyed buildings: https://data.humdata.org/dataset/hotosm_tur_destroyed_buildings 
- Maxar open data for pre disaster tiff files of satellite images: https://www.maxar.com/open-data
- QuickQuakeBuildings dataset already containing post disaster images and building footprints, also includes quad codes to obtain pre disaster images of same region and information on the pre defined folds: https://github.com/ya0-sun/PostEQ-SARopt-BuildingDamage?tab=readme-ov-file 





- the following code will limit the large geojson of building footprints to our area of interest. 
- This was done because the processing of the original geojson took too long.
- Only run this if you do not yet have the filtered geojson loaded! It will take a while.

In [ ]:
# geojson_path = "/Users/jonahvanemden/Desktop/thesis 2/hotosm_tur_buildings_polygons_geojson(1)/hotosm_tur_buildings_polygons_geojson.geojson"
# tif_folder = "/Users/jonahvanemden/Desktop/thesis 2/tif folder"

# buildings = gpd.read_file(geojson_path)  

# def get_tif_bounds(tif_path):
#     with rasterio.open(tif_path) as src:
#         return src.bounds  

# all_bounds = [get_tif_bounds(os.path.join(tif_folder, tif)) for tif in os.listdir(tif_folder) if tif.endswith(".tif")]

# min_x = min([b[0] for b in all_bounds])
# min_y = min([b[1] for b in all_bounds])
# max_x = max([b[2] for b in all_bounds])
# max_y = max([b[3] for b in all_bounds])

# combined_bounds = (min_x, min_y, max_x, max_y)

# sample_tif = os.path.join(tif_folder, [tif for tif in os.listdir(tif_folder) if tif.endswith(".tif")][0])
# with rasterio.open(sample_tif) as src:
#     tif_crs = src.crs

# buildings = buildings.to_crs(tif_crs)

# combined_box = box(*combined_bounds)


# filtered_buildings = buildings[buildings.intersects(combined_box)]  

# output_folder = "/Users/jonahvanemden/Desktop/thesis 2/hotosm_tur_buildings_polygons_geojson(1)"
# os.makedirs(output_folder, exist_ok=True)

# output_path = os.path.join(output_folder, "filtered_buildings.geojson")
# filtered_buildings.to_file(output_path, driver="GeoJSON")


# Inspect data

In [ ]:
base_folder = "/Users/jonahvanemden/Desktop/thesis 2/earthquake_building_dataset"  
damage_folder = os.path.join(base_folder, "damaged")
intact_folder = os.path.join(base_folder, "intact")

def get_unique_osm_ids(folder):
    filenames = [f for f in os.listdir(folder) if f.endswith(".mat")]
    osm_ids = set(f.split("_")[0] for f in filenames)
    return osm_ids

damage_osm_ids = get_unique_osm_ids(damage_folder)
intact_osm_ids = get_unique_osm_ids(intact_folder)

print(f"Buildings in damage: {len(damage_osm_ids)}")
print(f"Buildings in intact: {len(intact_osm_ids)}")

total_unique_osm_ids = damage_osm_ids.union(intact_osm_ids)
print(f"Total buildings: {len(total_unique_osm_ids)}")


# Get pre earthquake building patches, create shadow masks and load splits

In [3]:
geojson_path = "/Users/jonahvanemden/Desktop/thesis 2/hotosm_tur_buildings_polygons_geojson(1)/filtered_buildings.geojson"
tif_folder = "/Users/jonahvanemden/Desktop/thesis 2/tif folder"
output_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey earthquake patches"
damage_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/damaged"
intact_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/intact"
os.makedirs(output_folder, exist_ok=True)

def get_valid_osm_ids(folder):  # Make sure you only get images of the buildings of which there are post earthquake images.
    return set(f.split("_")[0] for f in os.listdir(folder) if f.endswith(".mat"))

valid_osm_ids = get_valid_osm_ids(damage_folder).union(get_valid_osm_ids(intact_folder))

buildings = gpd.read_file(geojson_path)

sample_tif = [os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")][0]
with rasterio.open(sample_tif) as src:  # Make sure coordinate reference system is correct.
    tif_crs = src.crs
    pixel_size = src.res[0]

buildings = buildings.to_crs(tif_crs)

tif_files = [os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")]

buffer_distance = 16 * pixel_size  # Buffer distance of 16 pixels surrounding the building to match the images in the original QQB dataset.
saved_count = 0

for i, row in tqdm(buildings.iterrows(), total=len(buildings), desc="Processing buildings", ncols=100):
    osm_id = str(row.get("osm_id", f"unknown_{i}"))

    if osm_id not in valid_osm_ids:
        continue

    geom = row['geometry'].buffer(buffer_distance)
    found = False

    for tif_path in tif_files:
        with rasterio.open(tif_path) as src:
            if not geom.intersects(box(*src.bounds)):
                continue 

            try:
                bbox = geom.bounds  
                window = rasterio.windows.from_bounds(*bbox, transform=src.transform)
                window = window.round_offsets().round_lengths()

                out_image = src.read(window=window)

                out_image[out_image == src.nodata] = 0

                out_image = np.moveaxis(out_image, 0, -1)
                out_image_uint8 = np.clip(out_image, 0, 255).astype(np.uint8)

                img = Image.fromarray(out_image_uint8)
                output_path = os.path.join(output_folder, f"{osm_id}_opt_pre.png")
                img.save(output_path)

                saved_count += 1
                found = True
                break

            except Exception as e:
                continue

    if not found:
        warnings.warn(f"Building {i} OSM ID: {osm_id} not found")


print(f"saved {saved_count} buildings")


Processing buildings: 100%|█████████████████████████████████████| 6799/6799 [19:09<00:00,  5.91it/s]

saved 3842 buildings


- Get pre earthquake building patches from destroyed buildings since they have a different geojson on HOTOSM but the process stays the same.

In [76]:
geojson_path = "/Users/jonahvanemden/Desktop/thesis 2/hotosm_tur_destroyed_buildings_polygons_geojson.geojson"
tif_folder = "/Users/jonahvanemden/Desktop/thesis 2/tif folder"
output_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey damaged patches"
damage_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/damaged"
intact_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/intact"
os.makedirs(output_folder, exist_ok=True)

def get_valid_osm_ids(folder):
    return set(f.split("_")[0] for f in os.listdir(folder) if f.endswith(".mat"))

valid_osm_ids = get_valid_osm_ids(damage_folder).union(get_valid_osm_ids(intact_folder))

buildings = gpd.read_file(geojson_path)

sample_tif = [os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")][0]
with rasterio.open(sample_tif) as src:
    tif_crs = src.crs
    pixel_size = src.res[0]

buildings = buildings.to_crs(tif_crs)

tif_files = [os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")]

buffer_distance = 16 * pixel_size
saved_count = 0

for i, row in tqdm(buildings.iterrows(), total=len(buildings), desc="Processing buildings", ncols=100):
    osm_id = str(row.get("osm_id", f"unknown_{i}"))

    if osm_id not in valid_osm_ids:
        continue

    geom = row['geometry'].buffer(buffer_distance)
    found = False

    for tif_path in tif_files:
        with rasterio.open(tif_path) as src:
            if not geom.intersects(box(*src.bounds)):
                continue  

            try:
               
                bbox = geom.bounds  
                window = rasterio.windows.from_bounds(*bbox, transform=src.transform)
                window = window.round_offsets().round_lengths()

                out_image = src.read(window=window)

                out_image[out_image == src.nodata] = 0

                out_image = np.moveaxis(out_image, 0, -1)
                out_image_uint8 = np.clip(out_image, 0, 255).astype(np.uint8)

                img = Image.fromarray(out_image_uint8)
                output_path = os.path.join(output_folder, f"{osm_id}_opt_pre.png")
                img.save(output_path)

                saved_count += 1
                found = True
                break

            except Exception as e:
                continue

    if not found:
        warnings.warn(f"Building {i} OSM ID: {osm_id} not found")


print(f"saved {saved_count} buildings")


Processing buildings: 100%|█████████████████████████████████████| 3239/3239 [00:50<00:00, 64.43it/s]

saved 169 buildings


- Check if buildings have not been saved in two folders

In [67]:
# I noticed that one building that was in the pre earthquake damaged folder was also saved in the pre earthquake intact folder.
# The following code finds that building and removes it from the intact folder since it was present in the post earthquake damaged folder.

base_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset"
damage_folder = os.path.join(base_folder, "damaged")
output_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey earthquake patches"

def get_unique_osm_ids(folder):
    filenames = [f for f in os.listdir(folder) if f.endswith(".mat")]
    osm_ids = set(f.split("_")[0] for f in filenames)
    return osm_ids

def get_saved_osm_ids(folder):
    filenames = [f for f in os.listdir(folder) if f.endswith("_opt_pre.png")]
    osm_ids = set(f.split("_")[0] for f in filenames)
    return osm_ids

damage_osm_ids = get_unique_osm_ids(damage_folder)
saved_osm_ids = get_saved_osm_ids(output_folder)

damage_saved_ids = saved_osm_ids.intersection(damage_osm_ids)

for osm_id in damage_saved_ids:
    print(osm_id)


In [ ]:
# Remove the image from the intact folder since it is in the damaged folder for post images from QQB.
output_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey earthquake patches"

osm_id_to_remove = list(damage_saved_ids)[0]  

filename = f"{osm_id_to_remove}_opt_pre.png"
file_path = os.path.join(output_folder, filename)

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Removed: {filename}")

- Get the shadow masks for the pre earthquake images

In [74]:
def threshold_shadow_segmentation(image):
    """
    Segment shadows in the image using Otsu's thresholding method.
    Shadows are converted to white (255), background to black (0).
    """
    gray_image = image.convert('L')
    gray_array = np.array(gray_image)

    _, shadow_mask = cv2.threshold(gray_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Apply Otsu's thresholding (shadows will be 0, background 255).

    shadow_mask = 255 - shadow_mask  # Invert the mask so shadows become white and background becomes black.

    return Image.fromarray(shadow_mask)


def process_shadow_masks(input_folder, output_folder):
    """
    Process all images in the input_folder to generate shadow masks.
    Saves the masks in the output_folder with filenames formatted as osm_id_pre_shadow.png.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in tqdm(os.listdir(input_folder), desc=f"Processing {os.path.basename(input_folder)} images"):
        if filename.endswith('.png'):
            osm_id = filename.split('_')[0]
            
            image_path = os.path.join(input_folder, filename)
        
            image = Image.open(image_path).convert('RGB')

            shadow_mask = threshold_shadow_segmentation(image)
            
            output_filename = f"{osm_id}_pre_shadow.png"
            output_path = os.path.join(output_folder, output_filename)
            shadow_mask.save(output_path)

damaged_images_folder = '/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey damaged patches'
intact_images_folder = '/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey earthquake patches'

damaged_output_folder = '/Users/jonahvanemden/Desktop/thesis 2/Data/pre shadows/Damaged'
intact_output_folder = '/Users/jonahvanemden/Desktop/thesis 2/Data/pre shadows/Intact'

process_shadow_masks(damaged_images_folder, damaged_output_folder)

process_shadow_masks(intact_images_folder, intact_output_folder)

Processing pre turkey earthquake patches images: 100%|██████████| 3841/3841 [00:10<00:00, 359.83it/s]


- Get shadow masks for post earthquake images

In [70]:
# The methodology for obtaining shadow masks for the post images is different since they are stored as .mat files.

def read_mat_image(mat_path):
    with h5py.File(mat_path, 'r') as f:
        key = list(f.keys())[0]
        array = np.array(f[key])
        if array.shape[0] == 3:
            array = np.transpose(array, (1, 2, 0))
        return array


def process_mat_folder(mat_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    processed_osm_ids = set()
    grouped_mats = {}

    for fname in sorted(os.listdir(mat_folder)):
        if not fname.endswith('_opt.mat'):
            continue
        osm_id = fname.split("_")[0]
        grouped_mats.setdefault(osm_id, []).append(fname)

    saved_count = 0

    for osm_id, files in tqdm(grouped_mats.items(), desc=f"Processing {os.path.basename(mat_folder)}", ncols=100):
        if osm_id in processed_osm_ids:
            continue

        mat_path = os.path.join(mat_folder, files[0])
        try:
            img_array = read_mat_image(mat_path)
            img_array = np.clip(img_array, 0, 255).astype(np.uint8)
            img_pil = Image.fromarray(img_array)

            shadow_mask = threshold_shadow_segmentation(img_pil)

            out_path = os.path.join(output_folder, f"{osm_id}_post_shadow.png")
            shadow_mask.save(out_path)

            processed_osm_ids.add(osm_id)
            saved_count += 1

        except Exception as e:
            print(f"{mat_path}: {e}")

    print(f"{output_folder}: {saved_count}")

damaged_post_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/damaged"
intact_post_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/intact"

damaged_output = "/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Damaged"
intact_output = "/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Intact"

process_mat_folder(damaged_post_folder, damaged_output)
process_mat_folder(intact_post_folder, intact_output)


Processing damaged: 100%|████████████████████████████████████████| 169/169 [00:00<00:00, 246.05it/s]


/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Damaged: 169


Processing intact: 100%|███████████████████████████████████████| 3860/3860 [00:13<00:00, 294.33it/s]

/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Intact: 3860


- Create splits linking all relevant images and information based on the pre defined splits from the QQB dataset. Only buildings that exist in the pre and post images are used. 

In [71]:
split_txt_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data split"
output_csv_folder = "/Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs"
os.makedirs(output_csv_folder, exist_ok=True)

folders = {
    0: { 
        "pre_optical": "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey earthquake patches",
        "pre_shadow": "/Users/jonahvanemden/Desktop/thesis 2/Data/pre shadows/Intact",
        "post": "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/intact",
        "post_shadow": "/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Intact",
    },
    1: {  
        "pre_optical": "/Users/jonahvanemden/Desktop/thesis 2/Data/pre earthquake data/pre turkey damaged patches",
        "pre_shadow": "/Users/jonahvanemden/Desktop/thesis 2/Data/pre shadows/Damaged",
        "post": "/Users/jonahvanemden/Desktop/thesis 2/Data/earthquake_building_dataset/damaged",
        "post_shadow": "/Users/jonahvanemden/Desktop/thesis 2/Data/post shadows/Damaged",
    }
}

def get_if_exists(path):
    return path if os.path.exists(path) else None

for split_filename in os.listdir(split_txt_folder):  # Process each split .txt file with pre defined 5 folds.
    if not split_filename.endswith(".txt"):
        continue

    split_path = os.path.join(split_txt_folder, split_filename)
    records = []

    with open(split_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if "," not in line:
            continue

        sample, label_str = line.split(",")
        label = int(label_str)
        osm_id = os.path.basename(sample)

        fldr = folders[label]

        paths = {
            "osm_id": osm_id,
            "label": label,
            "pre_optical": get_if_exists(os.path.join(fldr["pre_optical"], f"{osm_id}_opt_pre.png")),
            "pre_shadow": get_if_exists(os.path.join(fldr["pre_shadow"], f"{osm_id}_pre_shadow.png")),
            "post_optical_mat": get_if_exists(os.path.join(fldr["post"], f"{osm_id}_opt.mat")),
            "post_optical_footprint_mat": get_if_exists(os.path.join(fldr["post"], f"{osm_id}_optftp.mat")),
            "post_shadow": get_if_exists(os.path.join(fldr["post_shadow"], f"{osm_id}_post_shadow.png")),
        }

        if all(paths[k] for k in paths if k not in ["osm_id", "label"]):
            records.append(paths)

    output_path = os.path.join(output_csv_folder, split_filename.replace(".txt", ".csv"))
    pd.DataFrame(records).to_csv(output_path, index=False)
    print(f"{split_filename} {len(records)} samples saved to {output_path}")


fold-1.txt 801 samples saved to /Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs/fold-1.csv
fold-2.txt 804 samples saved to /Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs/fold-2.csv
fold-3.txt 803 samples saved to /Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs/fold-3.csv
fold-4.txt 802 samples saved to /Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs/fold-4.csv
fold-5.txt 800 samples saved to /Users/jonahvanemden/Desktop/thesis 2/Data/split_csvs/fold-5.csv
